# Module 5: Load features into Online Store InMemory Option
---

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.m5.large`.  This will provide the notebook more memory over the ml.t3.medium instance normally used. 


# Content
1. [Background](#Background)
1. [Setup](#Setup)
1. [Create Feature Group](#Create-Feature-Group)
1. [Read and Transform Orders data](#Read-and-Transform-Orders-Data)
1. [Write Orders Records to Online Store](#Write-Orders-Table-Records-to-Online-Store)
1. [Bulk Ingest Data to the Online Store (SageMaker Processing Job)](#Bulk-Ingest-Data-to-the-Online-Store)
1. [Read, Modify, and Write Records to Online Store](#Read,-Modify,-and-Write-Records-to-Online-Store)


# Background

Amazon SageMaker Feature Store now supports an [Online InMemory](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-storage-configurations-online-store.html#feature-store-storage-configurations-online-store-in-memory-tier) storage option to achieve very low latency access to feature data, which is backed by ElastiCache for Redis. This Online storage choice also supports additional [Collection data types](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-collection-types.html) such as LIST, SET, and VECTOR.

In this example notebook, we will use the SageMaker Feature Store runtime client to call `put_record` and `get_record` to write and read from the Online store. We will also demonstrate how customers can use the [Feature Store Spark Connector](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-ingestion-spark-connector-setup.html) to perform bulk ingestion of features directly to the Online InMemory store.

### Online-Only Feature Group

First, we will create a new Feature Store Feature Group with Online StorageType set to `InMemory`. This will configure the in-memory Feature Group and also disable the Offline storage.

Note: Currently (October 2023), the Online InMemory storage tier does *not* allow Offline storage option, and does not automatically replicate records to offline store.


# Setup

In [2]:
%%capture 

!pip install --upgrade sagemaker
!pip install --upgrade boto3

In [3]:
import sagemaker
import boto3

from time import gmtime, strftime, sleep
from random import randint

import pandas as pd
import numpy as np
import subprocess
import importlib
import logging
import time
import sys


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# Print SDK library versions
print(f'Boto3 version: {boto3.__version__}')
print(f'Sagemaker version: {sagemaker.__version__}')

Boto3 version: 1.28.62
Sagemaker version: 2.191.0


In [5]:
from sagemaker.feature_store.feature_definition import (
    FeatureDefinition, 
    StringFeatureDefinition, 
    FractionalFeatureDefinition, 
    FeatureTypeEnum, 
    CollectionType, 
    CollectionTypeEnum, 
    ListCollectionType
)
from sagemaker.feature_store.inputs import (
    FeatureValue,
    FeatureParameter,
    TableFormatEnum,
    Filter,
    ResourceEnum,
    Identifier,
    DeletionModeEnum,
    TtlDuration,
    OnlineStoreConfigUpdate,
    OnlineStoreStorageTypeEnum,
)
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_store import FeatureStore


In [6]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

### Essentials

In [7]:
sagemaker_session = sagemaker.session.Session()
region_name = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()
role_arn = sagemaker.get_execution_role()

print(region_name)
print(default_bucket)
print(role_arn)

sm_client = boto3.client('sagemaker')
fs_client = boto3.client('sagemaker-featurestore-runtime', region_name=region_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
us-west-2
sagemaker-us-west-2-572539092864
arn:aws:iam::572539092864:role/service-role/AmazonSageMaker-ExecutionRole-20200407T174741


In [8]:
# Define S3 prefixes for workshop notebook
feature_store_prefix = 'sagemaker-feature-store'
workshop_prefix = 'fscw'
s3_feature_store_workshop_prefix = f'{feature_store_prefix}/{workshop_prefix}'

print(f'S3 Workshop prefix: {s3_feature_store_workshop_prefix}')

s3_feature_store_data_prefix = f'{s3_feature_store_workshop_prefix}/data'
s3_uri_upload_prefix = f's3://{default_bucket}/{s3_feature_store_data_prefix}'
print(f'S3 URI Upload prefix: {s3_uri_upload_prefix}')

S3 Workshop prefix: sagemaker-feature-store/fscw
S3 URI Upload prefix: s3://sagemaker-us-west-2-572539092864/sagemaker-feature-store/fscw/data


### Append scripts directory to classpath

In [9]:
sys.path.append("./scripts")

In [10]:
from featurestore_helper import (
    get_order_features_with_list, 
    get_feature_definitions_with_list, 
    create_order_list_fg)

sample_feature_list = get_order_features_with_list()
sample_feature_list

[FeatureValue(feature_name='order_id', value_as_string='O999', value_as_string_list=None),
 FeatureValue(feature_name='customer_id', value_as_string='C999', value_as_string_list=None),
 FeatureValue(feature_name='product_id', value_as_string='P999', value_as_string_list=None),
 FeatureValue(feature_name='purchase_amount', value_as_string='9.99', value_as_string_list=None),
 FeatureValue(feature_name='is_reordered', value_as_string='1', value_as_string_list=None),
 FeatureValue(feature_name='event_time', value_as_string='2020-10-30T03:43:21Z', value_as_string_list=None),
 FeatureValue(feature_name='n_days_since_last_purchase', value_as_string='0.12209302325581398', value_as_string_list=None),
 FeatureValue(feature_name='n_days_list', value_as_string=None, value_as_string_list=['13', '27', '7', '11', '22'])]

In [11]:
# validate feature definitions used to create Feature Group
feature_definitions = get_feature_definitions_with_list()
feature_definitions

[StringFeatureDefinition(feature_name='order_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 StringFeatureDefinition(feature_name='customer_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 StringFeatureDefinition(feature_name='product_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FractionalFeatureDefinition(feature_name='purchase_amount', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 IntegralFeatureDefinition(feature_name='is_reordered', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 StringFeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FractionalFeatureDefinition(feature_name='n_days_since_last_purchase', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 StringFeatureDefinition(feature_name='n_days_list', feature_type=<FeatureTypeEn

# Create Feature Group

First, create a new Feature Group with Online storage enabled and Offline storage disabled. To use the In-Memory tier, hosted on AWS Elasticache/REDIS, set the StorageType to `InMemory`. To disable the Offline store, simply remove the `OfflineStoreConfig` from the configuration when calling `create_feature_group`. 

Here are the key configuration items for Online-only Store using InMemory option:

```
    OnlineStoreConfig={
      'EnableOnlineStore': True,
      'StorageType': 'InMemory',
    }
```

For more information, please refer to the OnlineStoreConfig [documentation](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_OnlineStoreConfig.html).

In [12]:
feature_group_name = 'FG-online-only-inmemory'

In [13]:
response = create_order_list_fg(feature_group_name, sagemaker_session, role_arn)
response

{'FeatureGroupArn': 'arn:aws:sagemaker:us-west-2:572539092864:feature-group/FG-online-only-inmemory',
 'ResponseMetadata': {'RequestId': 'b4ac7224-64a2-49ce-a036-ad2c13decf19',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b4ac7224-64a2-49ce-a036-ad2c13decf19',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Wed, 11 Oct 2023 18:17:19 GMT'},
  'RetryAttempts': 0}}

In [14]:
def wait_for_feature_group_creation_complete(feature_group_name):
    status = sm_client.describe_feature_group(FeatureGroupName=feature_group_name)['FeatureGroupStatus']
    print(f'Initial status: {status}')
    while status == 'Creating':
        logger.info(f'Waiting for feature group: {feature_group_name} to be created ...')
        time.sleep(60)
        status = sm_client.describe_feature_group(FeatureGroupName=feature_group_name)['FeatureGroupStatus']
    if status != 'Created':
        raise SystemExit(f'Failed to create feature group {feature_group_name}: {status}')
    logger.info(f'FeatureGroup {feature_group_name} was successfully created.')

In [15]:
wait_for_feature_group_creation_complete(feature_group_name)

Waiting for feature group: FG-online-only-inmemory to be created ...


Initial status: Creating


Waiting for feature group: FG-online-only-inmemory to be created ...
Waiting for feature group: FG-online-only-inmemory to be created ...
Waiting for feature group: FG-online-only-inmemory to be created ...
Waiting for feature group: FG-online-only-inmemory to be created ...
Waiting for feature group: FG-online-only-inmemory to be created ...
Waiting for feature group: FG-online-only-inmemory to be created ...
Waiting for feature group: FG-online-only-inmemory to be created ...
Waiting for feature group: FG-online-only-inmemory to be created ...
Waiting for feature group: FG-online-only-inmemory to be created ...
FeatureGroup FG-online-only-inmemory was successfully created.


### Describe Feature Group 

The Feature Group `describe` method returns all the metadata for the Feature Group, including the Feature Definitions which include name and data types. This essentially provides the schema for the Feature Group.

Note the presence of `CollectionType` in the output below.

In [16]:
# Describe for Feature Group

# Note the presence of 'CollectionType' in the Feature Definitions output below
fg = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)
fg.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-west-2:572539092864:feature-group/FG-online-only-inmemory',
 'FeatureGroupName': 'FG-online-only-inmemory',
 'RecordIdentifierFeatureName': 'order_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'order_id', 'FeatureType': 'String'},
  {'FeatureName': 'customer_id', 'FeatureType': 'String'},
  {'FeatureName': 'product_id', 'FeatureType': 'String'},
  {'FeatureName': 'purchase_amount', 'FeatureType': 'Fractional'},
  {'FeatureName': 'is_reordered', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'n_days_since_last_purchase', 'FeatureType': 'Fractional'},
  {'FeatureName': 'n_days_list',
   'FeatureType': 'String',
   'CollectionType': 'List'}],
 'CreationTime': datetime.datetime(2023, 10, 11, 18, 17, 19, 345000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True, 'StorageType': 'InMemory'},
 'RoleArn': 'arn:aws:iam::572539092864:role/servi

### Use sample record to test put_record and get_record

In [17]:
sample_feature_list

[FeatureValue(feature_name='order_id', value_as_string='O999', value_as_string_list=None),
 FeatureValue(feature_name='customer_id', value_as_string='C999', value_as_string_list=None),
 FeatureValue(feature_name='product_id', value_as_string='P999', value_as_string_list=None),
 FeatureValue(feature_name='purchase_amount', value_as_string='9.99', value_as_string_list=None),
 FeatureValue(feature_name='is_reordered', value_as_string='1', value_as_string_list=None),
 FeatureValue(feature_name='event_time', value_as_string='2020-10-30T03:43:21Z', value_as_string_list=None),
 FeatureValue(feature_name='n_days_since_last_purchase', value_as_string='0.12209302325581398', value_as_string_list=None),
 FeatureValue(feature_name='n_days_list', value_as_string=None, value_as_string_list=['13', '27', '7', '11', '22'])]

In [18]:
# Transform FeatureValue list to writable Record 

sample_record = []
for f in sample_feature_list:
    feature = {}
    feature['FeatureName'] = f.feature_name
    if (f.value_as_string_list):
        feature['ValueAsStringList'] = f.value_as_string_list
    else:
        feature['ValueAsString'] = f.value_as_string
    sample_record.append(feature)

# Dump sample record
sample_record

[{'FeatureName': 'order_id', 'ValueAsString': 'O999'},
 {'FeatureName': 'customer_id', 'ValueAsString': 'C999'},
 {'FeatureName': 'product_id', 'ValueAsString': 'P999'},
 {'FeatureName': 'purchase_amount', 'ValueAsString': '9.99'},
 {'FeatureName': 'is_reordered', 'ValueAsString': '1'},
 {'FeatureName': 'event_time', 'ValueAsString': '2020-10-30T03:43:21Z'},
 {'FeatureName': 'n_days_since_last_purchase',
  'ValueAsString': '0.12209302325581398'},
 {'FeatureName': 'n_days_list',
  'ValueAsStringList': ['13', '27', '7', '11', '22']}]

In [19]:
fs_client.put_record(FeatureGroupName=feature_group_name, Record=sample_record)

{'ResponseMetadata': {'RequestId': 'df95e0c9-cf7f-4d97-9ecb-8156b10faf77',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'df95e0c9-cf7f-4d97-9ecb-8156b10faf77',
   'content-type': 'application/json',
   'content-length': '0',
   'date': 'Wed, 11 Oct 2023 18:28:11 GMT'},
  'RetryAttempts': 0}}

In [20]:
sample_record_id = sample_record[0]['ValueAsString']
sample_record_id

'O999'

In [21]:
response = fs_client.get_record(FeatureGroupName=feature_group_name, 
                                RecordIdentifierValueAsString=sample_record_id)
response['Record']

[{'FeatureName': 'order_id', 'ValueAsString': 'O999'},
 {'FeatureName': 'customer_id', 'ValueAsString': 'C999'},
 {'FeatureName': 'product_id', 'ValueAsString': 'P999'},
 {'FeatureName': 'purchase_amount', 'ValueAsString': '9.99'},
 {'FeatureName': 'is_reordered', 'ValueAsString': '1'},
 {'FeatureName': 'event_time', 'ValueAsString': '2020-10-30T03:43:21Z'},
 {'FeatureName': 'n_days_since_last_purchase',
  'ValueAsString': '0.12209302325581398'},
 {'FeatureName': 'n_days_list',
  'ValueAsStringList': ['13', '27', '7', '11', '22']}]

# Read and Transform Orders Data

Read CSV file: We will read the "orders.csv" file from the local module 05 directory. We will then transform it to include a ListCollection type feature. You can read more about supported Collection Types in the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-collection-types.html).

In [22]:
orders_data_file = "../05-module-scalable-batch-ingestion/orders.csv"
orders_df = pd.read_csv(orders_data_file)

In [23]:
columns = orders_df.columns
print(columns)
print(orders_df.shape)

Index(['order_id', 'customer_id', 'product_id', 'purchase_amount',
       'is_reordered', 'event_time', 'n_days_since_last_purchase'],
      dtype='object')
(100000, 7)


In [24]:
# Function to generate list of days
def get_random_days_list(input):
    ndays = int(10.0 * input)
    days = []
    for day in range(0,ndays):
        n =randint(0, 30)
        days.append(str(n))
    return days


In [25]:
# Use Pandas map function to create new column data
new_days_col = orders_df['n_days_since_last_purchase'].map(lambda x: get_random_days_list(x))

print(new_days_col.size)

new_days_col.head(n=5)

100000


0                                   [7]
1    [23, 21, 22, 27, 8, 22, 5, 13, 14]
2                                    []
3                           [2, 26, 12]
4                               [29, 2]
Name: n_days_since_last_purchase, dtype: object

In [26]:
# Append new column with list values to Pandas dataframe
orders_list_df = orders_df
orders_list_df['n_days_list'] = new_days_col

print(orders_list_df.columns)
print(orders_list_df.shape)

Index(['order_id', 'customer_id', 'product_id', 'purchase_amount',
       'is_reordered', 'event_time', 'n_days_since_last_purchase',
       'n_days_list'],
      dtype='object')
(100000, 8)


In [27]:
orders_list_df.head(5)

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase,n_days_list
0,O1,C5731,P16,0.913465,1,2021-09-13T13:21:30.036Z,0.122093,[7]
1,O2,C3541,P12802,0.663168,1,2021-09-13T13:21:30.036Z,0.903101,"[23, 21, 22, 27, 8, 22, 5, 13, 14]"
2,O3,C7402,P8320,0.629604,1,2021-09-13T13:21:30.036Z,0.054264,[]
3,O4,C7356,P18000,0.202772,0,2021-09-13T13:21:30.036Z,0.343023,"[2, 26, 12]"
4,O5,C5806,P12940,0.053168,1,2021-09-13T13:21:30.036Z,0.242248,"[29, 2]"


In [28]:
# Dump one row of data
orders_list_df.iloc[1]

order_id                                                      O2
customer_id                                                C3541
product_id                                                P12802
purchase_amount                                         0.663168
is_reordered                                                   1
event_time                              2021-09-13T13:21:30.036Z
n_days_since_last_purchase                              0.903101
n_days_list                   [23, 21, 22, 27, 8, 22, 5, 13, 14]
Name: 1, dtype: object

In [29]:
orders_list_data_file = "../05-module-scalable-batch-ingestion/orders_list.csv"
orders_list_df.to_csv(orders_list_data_file, index=False)


In [30]:
# Upload orders_list data file to S3
s3_uri = sagemaker.s3.S3Uploader.upload(orders_list_data_file, s3_uri_upload_prefix)
s3_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


's3://sagemaker-us-west-2-572539092864/sagemaker-feature-store/fscw/data/orders_list.csv'

# Write Orders Table Records to Online Store

We now create a Python function to convert each row or orders data to a writeable feature record. Then we iterate over rows in the Pandas Dataframe and call put_record method to write to the Online-only Feature Group.

In [31]:
# Function to convert dataframe row data to feature for put_record
def convert_row_to_feature(row) -> list:
    record = []
    row_dict = row.to_dict()
    for col, val in row_dict.items():
        if isinstance(val, list):
            feature = {'FeatureName': col, 'ValueAsStringList': (val)}
        else:
            feature = {'FeatureName': col, 'ValueAsString': str(val)}
        record.append(feature)
    return record

In [32]:
%%time

# Iterate through dataframe rows and call put_record to write list data
for idx, row in orders_list_df.iterrows():
    rec = convert_row_to_feature(row)
    try:
        fs_client.put_record(FeatureGroupName=feature_group_name, Record=rec)
    except Exception:
        logger.error(f'Error calling put_record for row # {idx}')
    # Only write 10,000 rows to Feature Store from notebook
    if (idx >= 10000): break


CPU times: user 13.5 s, sys: 421 ms, total: 13.9 s
Wall time: 47 s


# Bulk Ingest Data to the Online Store

We will create a [SageMaker Processing Job](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html) which uses the Feature Store Spark Connector to ingest feature data from a Spark Dataframe directly into the online store.

To use the Feature Store Spark Connector in a Processing Job, we recommend extending the prebuilt SageMaker Spark Processing container as shown in the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-ingestion-spark-connector-setup.html#:~:text=Installation%20on%20a%20Amazon%20SageMaker%20Processing%20Job
). 

For this example, we will install the Spark Connector to a local directory and submit the required modules and Jar file when we run the processing job.

### Prepare Feature Store Pyspark library

The `sagemaker_feature_store_pyspark` library is available on github and can be used in SageMaker Studio notebooks by installing with `pip`. This library implements a Spark connector to Feature Store and enables extra functionality. We will be using the `FeatureStoreManager` class to ingest records to the Online-only InMemory store.

In [33]:
spark_version = '3.1' # MAJOR.MINOR

Install the Spark Connector under `./temp`.

In [34]:
%pip install sagemaker-feature-store-pyspark-{spark_version} -t ./temp --no-binary :all:

  Using cached sagemaker_feature_store_pyspark_3.1-1.1.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


Zip up the required Python modules.

In [35]:
import zipfile
import os

zf = zipfile.ZipFile('feature_store_pyspark.zip', 'w', zipfile.ZIP_DEFLATED)

for f in os.listdir('./temp/feature_store_pyspark'):
    if f.endswith('.py'):
        zf.write(os.path.join('./temp/feature_store_pyspark', f), os.path.join('feature_store_pyspark', f))

zf.close()

Use `feature_store_pyspark.classpath_jars()` to get the absolute path to the Jar file.

In [36]:
from temp import feature_store_pyspark

jar_path = feature_store_pyspark.classpath_jars()[0]
jar_path

'/root/Workshop/amazon-sagemaker-feature-store-end-to-end-workshop/05-module-scalable-batch-ingestion/temp/feature_store_pyspark/jars/sagemaker-feature-store-spark-sdk.jar'

In [37]:
# Upload original 'orders.csv' data file to S3 for Bulk Ingest Job
s3_uri_full_csv_path = sagemaker.s3.S3Uploader.upload(orders_data_file, s3_uri_upload_prefix)

print(f'\nUploaded CSV file to S3 location: \n{s3_uri_full_csv_path}')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml

Uploaded CSV file to S3 location: 
s3://sagemaker-us-west-2-572539092864/sagemaker-feature-store/fscw/data/orders.csv


Run a processing job using `scripts/ingest_to_online_store.py` and include the zipped Python modules and Jar file.

In [ ]:
from sagemaker.spark.processing import PySparkProcessor

spark_processor = PySparkProcessor(
    base_job_name='sm-processing-pyspark-fs-ingestion',
    framework_version=spark_version,
    role=role_arn,
    instance_count=1,
    instance_type='ml.m5.large',
    max_runtime_in_seconds=1200, 
    env={'AWS_DEFAULT_REGION': boto3.Session().region_name, 'mode': 'python'}
)

spark_processor.run(
    submit_app='./scripts/ingest_to_online_store.py',
    arguments=[
        '--feature_group_name', feature_group_name,
        '--region_name', region_name,
        '--s3_uri_csv_path', s3_uri_full_csv_path
    ],
    logs=False,
    submit_jars=[jar_path],
    submit_py_files=[
        './feature_store_pyspark.zip'
    ]
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker:Creating processing-job with name sm-processing-pyspark-fs-ingestion-2023-10-11-18-31-20-751


................................................................................................!

# Read, Modify, and Write Records to Online Store

Next, we verify that orders data is available in the Online store. Then we read, modify, and write back a record from Online store.

In [39]:
# Read one record from Online InMemory Store
response = fs_client.get_record(
    FeatureGroupName=feature_group_name,
    RecordIdentifierValueAsString='O1'
)
record = response['Record']
record

[{'FeatureName': 'order_id', 'ValueAsString': 'O1'},
 {'FeatureName': 'customer_id', 'ValueAsString': 'C5731'},
 {'FeatureName': 'product_id', 'ValueAsString': 'P16'},
 {'FeatureName': 'purchase_amount', 'ValueAsString': '0.9134653465346535'},
 {'FeatureName': 'is_reordered', 'ValueAsString': '1'},
 {'FeatureName': 'event_time', 'ValueAsString': '2021-09-13T13:21:30.036Z'},
 {'FeatureName': 'n_days_since_last_purchase',
  'ValueAsString': '0.12209302325581398'}]

### Modify and write the record back to Online store

In [40]:
# Modify two fields of retrieved orders record 
for feature in record:
    if feature['FeatureName'] == 'purchase_amount':
        feature['ValueAsString'] = '99.99'
    if feature['FeatureName'] == 'customer_id':
        feature['ValueAsString'] = 'C9999'


In [41]:
# Write updated record back to Online store
fs_client.put_record(FeatureGroupName=feature_group_name, Record=record)

{'ResponseMetadata': {'RequestId': 'aff7c04c-afa0-4b63-8ac8-530a43635f1d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aff7c04c-afa0-4b63-8ac8-530a43635f1d',
   'content-type': 'application/json',
   'content-length': '0',
   'date': 'Wed, 11 Oct 2023 18:44:10 GMT'},
  'RetryAttempts': 0}}

Verify that the latest feature data is available in the online store.

In [42]:
response = fs_client.get_record(
    FeatureGroupName=feature_group_name,
    RecordIdentifierValueAsString='O1'
)
record = response['Record']
record

[{'FeatureName': 'order_id', 'ValueAsString': 'O1'},
 {'FeatureName': 'customer_id', 'ValueAsString': 'C9999'},
 {'FeatureName': 'product_id', 'ValueAsString': 'P16'},
 {'FeatureName': 'purchase_amount', 'ValueAsString': '99.99'},
 {'FeatureName': 'is_reordered', 'ValueAsString': '1'},
 {'FeatureName': 'event_time', 'ValueAsString': '2021-09-13T13:21:30.036Z'},
 {'FeatureName': 'n_days_since_last_purchase',
  'ValueAsString': '0.12209302325581398'}]

### Use Batch Get Record call to test retrieval of multiple records from the online store.

In [43]:
# Call Batch Get Record to read multiple records from Online store
fs_client.batch_get_record(
    Identifiers=[{
        'FeatureGroupName': feature_group_name,
        'RecordIdentifiersValueAsString': ['O1', 'O2', 'O3', 'O4', 'O5']
    }]
)

{'ResponseMetadata': {'RequestId': '60cb88d1-9fe2-41d5-a105-70f3b91d8f29',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '60cb88d1-9fe2-41d5-a105-70f3b91d8f29',
   'content-type': 'application/json',
   'content-length': '3636',
   'date': 'Wed, 11 Oct 2023 18:44:23 GMT'},
  'RetryAttempts': 0},
 'Records': [{'FeatureGroupName': 'FG-online-only-inmemory',
   'RecordIdentifierValueAsString': 'O2',
   'Record': [{'FeatureName': 'order_id', 'ValueAsString': 'O2'},
    {'FeatureName': 'customer_id', 'ValueAsString': 'C3541'},
    {'FeatureName': 'product_id', 'ValueAsString': 'P12802'},
    {'FeatureName': 'purchase_amount', 'ValueAsString': '0.6631683168316832'},
    {'FeatureName': 'is_reordered', 'ValueAsString': '1'},
    {'FeatureName': 'event_time', 'ValueAsString': '2021-09-13T13:21:30.036Z'},
    {'FeatureName': 'n_days_since_last_purchase',
     'ValueAsString': '0.9031007751937984'}]},
  {'FeatureGroupName': 'FG-online-only-inmemory',
   'RecordIdentifierValueAsS

# Cleanup (optional)

To delete the Feature Group created in this notebook, uncomment and run the code below

In [ ]:
# Delete Feature Group, Online-Only storage

#response = sm_client.delete_feature_group(FeatureGroupName=feature_group_name)
#response